In [ ]:
!pip install torchmetrics

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 806.1/806.1 kB 6.6 MB/s eta 0:00:00


In [ ]:
import torch
from torch.utils.data import TensorDataset, DataLoader, RandomSampler, SequentialSampler
from transformers import BertTokenizer, BertForSequenceClassification
from transformers import GPT2Tokenizer, GPT2ForSequenceClassification, GPT2Config
from sklearn.model_selection import train_test_split

import pandas as pd
import numpy as np

from tabulate import tabulate
from tqdm import trange
import random
from torchmetrics.classification import Recall, Accuracy, AUROC, Precision

In [ ]:
FILL = "FILL"

In [ ]:
!wget 'https://archive.ics.uci.edu/ml/machine-learning-databases/00228/smsspamcollection.zip'
!unzip -o smsspamcollection.zip

--2023-12-03 14:27:14--  https://archive.ics.uci.edu/ml/machine-learning-databases/00228/smsspamcollection.zip
Resolving archive.ics.uci.edu (archive.ics.uci.edu)... 128.195.10.252
Connecting to archive.ics.uci.edu (archive.ics.uci.edu)|128.195.10.252|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: unspecified
Saving to: ‘smsspamcollection.zip’

smsspamcollection.z     [ <=>                ] 198.65K  --.-KB/s    in 0.1s    

2023-12-03 14:27:15 (1.81 MB/s) - ‘smsspamcollection.zip’ saved [203415]

Archive:  smsspamcollection.zip
  inflating: SMSSpamCollection       
  inflating: readme                  


In [ ]:
!unzip -o smsspamcollection.zip

Archive:  smsspamcollection.zip
  inflating: SMSSpamCollection       
  inflating: readme                  


In [ ]:
!head -10 SMSSpamCollection

ham	Go until jurong point, crazy.. Available only in bugis n great world la e buffet... Cine there got amore wat...
ham	Ok lar... Joking wif u oni...
spam	Free entry in 2 a wkly comp to win FA Cup final tkts 21st May 2005. Text FA to 87121 to receive entry question(std txt rate)T&C's apply 08452810075over18's
ham	U dun say so early hor... U c already then say...
ham	Nah I don't think he goes to usf, he lives around here though
spam	FreeMsg Hey there darling it's been 3 week's now and no word back! I'd like some fun you up for it still? Tb ok! XxX std chgs to send, £1.50 to rcv
ham	Even my brother is not like to speak with me. They treat me like aids patent.
ham	As per your request 'Melle Melle (Oru Minnaminunginte Nurungu Vettam)' has been set as your callertune for all Callers. Press *9 to copy your friends Callertune
spam	WINNER!! As a valued network customer you have been selected to receivea £900 prize reward! To claim call 09061701461. Claim code KL341. Valid 12 hours only.
spam	H

In [ ]:
!wget 'https://archive.ics.uci.edu/ml/machine-learning-databases/00228/smsspamcollection.zip'
!unzip -o smsspamcollection.zip

--2023-12-03 14:27:15--  https://archive.ics.uci.edu/ml/machine-learning-databases/00228/smsspamcollection.zip
Resolving archive.ics.uci.edu (archive.ics.uci.edu)... 128.195.10.252
Connecting to archive.ics.uci.edu (archive.ics.uci.edu)|128.195.10.252|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: unspecified
Saving to: ‘smsspamcollection.zip.1’

smsspamcollection.z     [ <=>                ] 198.65K  --.-KB/s    in 0.1s    

2023-12-03 14:27:15 (1.83 MB/s) - ‘smsspamcollection.zip.1’ saved [203415]

Archive:  smsspamcollection.zip
  inflating: SMSSpamCollection       
  inflating: readme                  


In [ ]:
file_path = 'SMSSpamCollection'
df = pd.DataFrame({'label':int(), 'text':str()}, index = [])
with open(file_path) as f:
    for line in f.readlines():
        split = line.split('\t')
        df = pd.concat([
                df,
                pd.DataFrame.from_dict({
                    'label': [1 if split[0] == 'spam' else 0],
                    'text': [split[1]]
                })
            ],
            ignore_index=True
        )
df.head()

,label,text
0,0,"Go until jurong point, crazy.. Available only ..."
1,0,Ok lar... Joking wif u oni...\n
2,1,Free entry in 2 a wkly comp to win FA Cup fina...
3,0,U dun say so early hor... U c already then say...
4,0,"Nah I don't think he goes to usf, he lives aro..."


In [ ]:
text = df.text.values
labels = df.label.values

In [ ]:
# Set to the GPT2Tokenizer and set lower case to True
tokenizer = FILL
tokenizer.pad_token = FILL
# Set the padding to 'left' or 'right'?
# Remember we want to use the last token's embedding to represent the entire sentence
tokenizer.padding_side = FILL

vocab.json:   0%|          | 0.00/1.04M [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/456k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.36M [00:00<?, ?B/s]

config.json:   0%|          | 0.00/665 [00:00<?, ?B/s]

In [ ]:
def print_rand_sentence():
    '''Displays the tokens and respective IDs of a random text sample'''
    index = random.randint(0, len(text)-1)
    print(text[index])
    table = np.array([tokenizer.tokenize(text[index]),
                    tokenizer.convert_tokens_to_ids(tokenizer.tokenize(text[index]))]).T
    print(tabulate(table,
                 headers = ['Tokens', 'Token IDs'],
                 tablefmt = 'fancy_grid'))

print_rand_sentence()

Oh k k:)but he is not a big hitter.anyway good

╒══════════╤═════════════╕
│ Tokens   │   Token IDs │
╞══════════╪═════════════╡
│ Oh       │        5812 │
├──────────┼─────────────┤
│ Ġk       │         479 │
├──────────┼─────────────┤
│ Ġk       │         479 │
├──────────┼─────────────┤
│ :        │          25 │
├──────────┼─────────────┤
│ )        │           8 │
├──────────┼─────────────┤
│ but      │        4360 │
├──────────┼─────────────┤
│ Ġhe      │         339 │
├──────────┼─────────────┤
│ Ġis      │         318 │
├──────────┼─────────────┤
│ Ġnot     │         407 │
├──────────┼─────────────┤
│ Ġa       │         257 │
├──────────┼─────────────┤
│ Ġbig     │        1263 │
├──────────┼─────────────┤
│ Ġhitter  │       30858 │
├──────────┼─────────────┤
│ .        │          13 │
├──────────┼─────────────┤
│ any      │        1092 │
├──────────┼─────────────┤
│ way      │        1014 │
├──────────┼─────────────┤
│ Ġgood    │         922 │
├──────────┼─────────────┤
│ Ċ    

In [ ]:
token_id = []
attention_masks = []

def preprocessing(input_text, tokenizer):
  '''
  Returns <class transformers.tokenization_utils_base.BatchEncoding> with the following fields:
    - input_ids: list of token ids
    - token_type_ids: list of token type ids
    - attention_mask: list of indices (0,1) specifying which tokens should considered by the model (return_attention_mask = True).
  '''
  # Use the tokenizer and the encode_plus methods to return the right data we'll need
  # Set max_length = 32 and return_tokens = 'pt'
  # Set other fields to the appropriate booleans needed
  return FILL


for sample in text:
    encoding_dict = preprocessing(sample, tokenizer)
    token_id.append(encoding_dict['input_ids'])
    attention_masks.append(encoding_dict['attention_mask'])


# Gather all the torch_id, attention masks, and labels
token_id = FILL
attention_masks = FILL
labels = FILL

Truncation was not explicitly activated but `max_length` is provided a specific value, please use `truncation=True` to explicitly truncate examples to max length. Defaulting to 'longest_first' truncation strategy. If you encode pairs of sequences (GLUE-style) with the tokenizer you can select this strategy more precisely by providing a specific strategy to `truncation`.
/usr/local/lib/python3.10/dist-packages/transformers/tokenization_utils_base.py:2614: FutureWarning: The `pad_to_max_length` argument is deprecated and will be removed in a future version, use `padding=True` or `padding='longest'` to pad to the longest sequence in the batch, or use `padding='max_length'` to pad to a max length. In this case, you can give a specific length with `max_length` (e.g. `max_length=45`) or leave max_length to None to pad to the maximal input size of the model (e.g. 512 for Bert).
  warnings.warn(


In [ ]:
def print_rand_sentence_encoding():
    '''Displays tokens, token IDs and attention mask of a random text sample'''
    index = random.randint(0, len(text) - 1)
    tokens = tokenizer.tokenize(tokenizer.decode(token_id[index]))
    print(tokens)
    token_ids = [i.numpy() for i in token_id[index]]
    attention = [i.numpy() for i in attention_masks[index]]
    table = np.array([tokens, token_ids, attention]).T
    print(
        tabulate(
            table,
            headers = ['Tokens', 'Token IDs', 'Attention Mask'],
            tablefmt = 'fancy_grid')
    )

print_rand_sentence_encoding()

['<|endoftext|>', '<|endoftext|>', '<|endoftext|>', '<|endoftext|>', '<|endoftext|>', '<|endoftext|>', '<|endoftext|>', '<|endoftext|>', '<|endoftext|>', '<|endoftext|>', '<|endoftext|>', '<|endoftext|>', '<|endoftext|>', '<|endoftext|>', '<|endoftext|>', '<|endoftext|>', 'She', 'Ġran', 'Ġoff', 'Ġwith', 'Ġa', 'Ġyounger', 'Ġman', '.', 'Ġwe', 'Ġwill', 'Ġmake', 'Ġpretty', 'Ġbabies', 'Ġtogether', 'Ġ:)', 'Ċ']
╒═══════════════╤═════════════╤══════════════════╕
│ Tokens        │   Token IDs │   Attention Mask │
╞═══════════════╪═════════════╪══════════════════╡
│ <|endoftext|> │       50256 │                0 │
├───────────────┼─────────────┼──────────────────┤
│ <|endoftext|> │       50256 │                0 │
├───────────────┼─────────────┼──────────────────┤
│ <|endoftext|> │       50256 │                0 │
├───────────────┼─────────────┼──────────────────┤
│ <|endoftext|> │       50256 │                0 │
├───────────────┼─────────────┼──────────────────┤
│ <|endoftext|> │       50256 │

In [ ]:
val_ratio = 0.2
# Recommended batch size: 16, 32. See: https://arxiv.org/pdf/1810.04805.pdf
batch_size = 16

# Indices of the train and validation splits stratified by labels
# Use train_test_split
train_idx, val_idx = FILL

# Train and validation sets
# Set to TensorDataset
train_set = FILL

val_set = FILL

# Prepare DataLoader
train_dataloader = FILL

validation_dataloader = FILL

### Load specific versions of the model

In [ ]:
# Load the BertForSequenceClassification model
# Do not ouput the attentions and all hidden states

config = FILL

# Set to 'gpt2' (the smallest GPT2 which is 120 M parameters)
# Use the config above and set other labels as needed
model = FILL

# Set the pad token id to the eos token id
model.config.pad_token_id = FILL

# Recommended learning rates (Adam): 5e-5, 3e-5, 2e-5
# See: https://arxiv.org/pdf/1810.04805.pdf
optimizer = torch.optim.AdamW(
    FILL,
    lr = 5e-5,
    eps = 1e-08
)

Some weights of GPT2ForSequenceClassification were not initialized from the model checkpoint at gpt2 and are newly initialized: ['score.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


### Set the model to the right device

In [ ]:
# If on GPU, do as below
device = FULL

In [ ]:
model = model.to(device)

# Recommended number of epochs: See: https://arxiv.org/pdf/1810.04805.pdf
epochs = 2

In [ ]:
# Print all the layers of this GPT2 model and the number of parameters per layer
# If this is False, fine tune just the classifier layer and leave all other GPT2 parameters alone
# If this is True, fine tune everything
fine_tune = True

# Turn off gradients using the above
FILL

assert(total_parameters == 124441344)

transformer.wte.weight 38597376 torch.Size([50257, 768]) True
transformer.wpe.weight 786432 torch.Size([1024, 768]) True
transformer.h.0.ln_1.weight 768 torch.Size([768]) True
transformer.h.0.ln_1.bias 768 torch.Size([768]) True
transformer.h.0.attn.c_attn.weight 1769472 torch.Size([768, 2304]) True
transformer.h.0.attn.c_attn.bias 2304 torch.Size([2304]) True
transformer.h.0.attn.c_proj.weight 589824 torch.Size([768, 768]) True
transformer.h.0.attn.c_proj.bias 768 torch.Size([768]) True
transformer.h.0.ln_2.weight 768 torch.Size([768]) True
transformer.h.0.ln_2.bias 768 torch.Size([768]) True
transformer.h.0.mlp.c_fc.weight 2359296 torch.Size([768, 3072]) True
transformer.h.0.mlp.c_fc.bias 3072 torch.Size([3072]) True
transformer.h.0.mlp.c_proj.weight 2359296 torch.Size([3072, 768]) True
transformer.h.0.mlp.c_proj.bias 768 torch.Size([768]) True
transformer.h.1.ln_1.weight 768 torch.Size([768]) True
transformer.h.1.ln_1.bias 768 torch.Size([768]) True
transformer.h.1.attn.c_attn.weigh

### Train the model

In [ ]:
# Use torchmetrics to set up accuracy, recall, precision, and auroc
accuracy = FILL
recall = FILL
precision = FILL
auroc = FILL

In [ ]:
# Main training / validation loop
for _ in trange(epochs, desc = 'Epoch'):

    # ========== Training ==========

    # Set model to training mode
    FILL

    # Tracking variables
    tr_loss = 0
    nb_tr_examples, nb_tr_steps = 0, 0

    for step, batch in enumerate(train_dataloader):
        # Put each element of batch onto the device
        batch = FILL

        # Unpack the batch
        FILL

        # Set gradients to zero
        FILL

        # Forward pass
        train_output = FILL

        # Backward pass
        FILL
        FILL

        # Update tracking variables
        tr_loss += FILL
        nb_tr_examples += FILL
        nb_tr_steps += FILL

    # ========== Validation ==========

    # Set model to evaluation mode
    FILL

    # Tracking variables
    val_accuracy = []
    val_precision = []
    val_recall = []
    val_auroc = []

    for batch in validation_dataloader:
        batch = FILL

        # Unpack the batch
        FILL

        with torch.no_grad():
          # Forward pass
            eval_output = FILL

        # Calculate validation metrics
        labels = FILL
        predicted_labels = FILL

        val_accuracy.append(FILL)
        val_recall.append(FILL)
        val_precision.append(FILL)
        val_auroc.append(FILL)

    print('\n\t - Train loss: {:.4f}'.format(tr_loss / nb_tr_steps))
    print('\t - Validation Accuracy: {:.4f}'.format(sum(val_accuracy)/len(val_accuracy)))
    print('\t - Validation Precision: {:.4f}'.format(sum(val_precision)/len(val_precision)))
    print('\t - Validation Recall: {:.4f}'.format(sum(val_recall)/len(val_recall)))
    print('\t - Validation AUROC: {:.4f}\n'.format(sum(val_auroc)/len(val_auroc)))

Epoch:   0%|          | 0/2 [00:00<?, ?it/s]/usr/local/lib/python3.10/dist-packages/torchmetrics/utilities/prints.py:43: UserWarning: No positive samples in targets, true positive value should be meaningless. Returning zero tensor in true positive score
  warnings.warn(*args, **kwargs)  # noqa: B028
Epoch:  50%|█████     | 1/2 [31:09<31:09, 1869.30s/it]


	 - Train loss: 0.1321
	 - Validation Accuracy: 0.9911
	 - Validation Precision: 0.9357
	 - Validation Recall: 0.8964
	 - Validation AUROC: 0.9263



Epoch: 100%|██████████| 2/2 [1:01:45<00:00, 1852.70s/it]


	 - Train loss: 0.0426
	 - Validation Accuracy: 0.9938
	 - Validation Precision: 0.9381
	 - Validation Recall: 0.9310
	 - Validation AUROC: 0.9426



### Test on a specific sentence, see the outcome

In [ ]:
new_sentence = 'WINNER!! As a valued network customer you have been selected to receivea £900 prize reward! To claim call 09061701461. Claim code KL341. Valid 12 hours only.'

# We need Token IDs and Attention Mask for inference on the new sentence
test_ids = []
test_attention_mask = []

# Apply the tokenizer
encoding = preprocessing(new_sentence, tokenizer)

# Extract IDs and Attention Mask
test_ids.append(encoding['input_ids'])
test_attention_mask.append(encoding['attention_mask'])
test_ids = torch.cat(test_ids, dim = 0)
test_attention_mask = torch.cat(test_attention_mask, dim = 0)

# Forward pass, calculate logit predictions
with torch.no_grad():
    output = model(test_ids.to(device), token_type_ids = None, attention_mask = test_attention_mask.to(device))

prediction = 'Spam' if np.argmax(output.logits.cpu().numpy()).flatten().item() == 1 else 'Ham'

print('Input Sentence: ', new_sentence)
print('Predicted Class: ', prediction)

### Questions

Question 1: Run the above by fine tuning GPT2 and the classfier head and by not doing this (using GPT2 as a feature encoder). What is the gap between this? What are the metrics we get in each case?

Solution: FILL Paste your output here